# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)  
X = df['message']
Y = df[df.columns[4::]]
Y = np.array(Y)

In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, y_train)
# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
#Generate classification report
def generate_report(y_test,y_pred,estimator):
    columns = df.columns[4::]
    accuracy_dict = {}
    
    #Generate and print report
    for i,column in enumerate(columns):
        y_test_column = y_test[:,i]
        y_pred_column = y_pred[:,i]
        num_classes = y_test[:,i].max() + 1
        accuracy = accuracy_score(y_test_column, y_pred_column)
        accuracy_dict[column] = {'accuracy_'+estimator:round(accuracy,2)}
        if num_classes == 3: 
            target_names = [column+'_0', column+'_1', column+'_2']      

        elif num_classes == 2: 
            target_names = [column+'_0', column+'_1']

        elif num_classes == 1: 
            target_names = [column+'_0']

        print(classification_report(y_test_column, y_pred_column, target_names=target_names))
        
    #Tabulate accuracy for classes
    df_accuracy = pd.DataFrame(accuracy_dict)
    df_accuracy = df_accuracy.T
       
    return df_accuracy

df_accuracy = generate_report(y_test,y_pred,'KNeighbors')
print(df_accuracy)

             precision    recall  f1-score   support

  related_0       0.55      0.43      0.48      1498
  related_1       0.84      0.89      0.86      5011
  related_2       0.47      0.20      0.28        45

avg / total       0.77      0.78      0.77      6554

             precision    recall  f1-score   support

  request_0       0.88      0.98      0.93      5433
  request_1       0.79      0.35      0.48      1121

avg / total       0.86      0.87      0.85      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6512
    offer_1       0.00      0.00      0.00        42

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.65      0.90      0.76      3773
aid_related_1       0.72      0.34      0.46      2781

  avg / total       0.68      0.66      0.63      6554

                precision    recall  f1-score   support

medical_help_0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f9763f76378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
#List Hyperparameters that we want to tune.
parameters = {
#    'vect__ngram_range': ((1, 1), (1, 2)),
#    'vect__max_df': (0.5, 0.75, 1.0),
#    'vect__max_features': (None, 5000, 10000),
#   'clf__estimator__leaf_size': [10,30,50],
    'clf__estimator__n_neighbors': [5,10,20],
#    'clf__estimator__p': [1,2]
} 
pipeline_tune = pipeline
pipeline_tune = GridSearchCV(pipeline_tune, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline_tune.fit(X_train, y_train)
# predict on test data
y_pred = pipeline_tune.predict(X_test)

df_accuracy2 = generate_report(y_test,y_pred,'KNeighbors_tune')
df_compare = pd.concat([df_accuracy,df_accuracy2],axis=1)
df_compare

             precision    recall  f1-score   support

  related_0       0.65      0.37      0.47      1553
  related_1       0.82      0.94      0.88      4958
  related_2       0.80      0.09      0.17        43

avg / total       0.78      0.80      0.77      6554

             precision    recall  f1-score   support

  request_0       0.90      0.97      0.93      5428
  request_1       0.76      0.50      0.60      1126

avg / total       0.88      0.89      0.88      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6516
    offer_1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.72      0.80      0.76      3821
aid_related_1       0.67      0.55      0.60      2733

  avg / total       0.70      0.70      0.69      6554

                precision    recall  f1-score   support

medical_help_0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy_KNeighbors,accuracy_KNeighbors_tune
related,0.78,0.80
request,0.87,0.89
offer,0.99,0.99
aid_related,0.66,0.70
medical_help,0.91,0.89
medical_products,0.95,0.94
search_and_rescue,0.97,0.97
security,0.98,0.98
military,0.97,0.96
child_alone,1.00,1.00


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
#Try Random forest classifier
from sklearn.ensemble import RandomForestClassifier
pipeline_Randomforest = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline_Randomforest.fit(X_train, y_train)
# predict on test data
y_pred = pipeline_Randomforest.predict(X_test)
df_accuracy3 = generate_report(y_test,y_pred,'RandomForest')
df_compare = pd.concat([df_accuracy,df_accuracy2,df_accuracy3],axis=1)
df_compare

             precision    recall  f1-score   support

  related_0       0.62      0.35      0.45      1505
  related_1       0.82      0.93      0.87      5006
  related_2       0.44      0.19      0.26        43

avg / total       0.77      0.79      0.77      6554

             precision    recall  f1-score   support

  request_0       0.88      0.98      0.93      5418
  request_1       0.83      0.38      0.52      1136

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6516
    offer_1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.73      0.87      0.79      3819
aid_related_1       0.76      0.55      0.63      2735

  avg / total       0.74      0.74      0.73      6554

                precision    recall  f1-score   support

medical_help_0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy_KNeighbors,accuracy_KNeighbors_tune,accuracy_RandomForest
related,0.78,0.80,0.79
request,0.87,0.89,0.88
offer,0.99,0.99,0.99
aid_related,0.66,0.70,0.74
medical_help,0.91,0.89,0.92
medical_products,0.95,0.94,0.95
search_and_rescue,0.97,0.97,0.97
security,0.98,0.98,0.98
military,0.97,0.96,0.97
child_alone,1.00,1.00,1.00


In [21]:
for model in df_compare.columns:
    mean = df_compare[model].mean()
    print('{} mean: {}'.format(model, mean))

accuracy_KNeighbors mean: 0.928888888888889
accuracy_KNeighbors_tune mean: 0.9325
accuracy_RandomForest mean: 0.9416666666666667


Random Forest model has the best accuracy.

### 9. Export your model as a pickle file

In [22]:
import pickle
filename = 'model.pkl'
pickle.dump(pipeline_Randomforest, open(filename, 'wb'))

In [23]:
X_test.iloc[1111]

'They were also due to discuss developments such as the recent terrorist act on a US warship in Yemen.'

In [24]:
pipeline_Randomforest.predict([X_test.iloc[1111]])

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [26]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.predict([X_test.iloc[1111]])

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.